In [ ]:
import os
import geopandas as gpd
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
model_name = "Sandrro/emotions_classificator"

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Создаем пайплайн для классификации
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

In [ ]:
def classify_text(text):
    try:
        # Убедимся, что текст является строкой
        if not isinstance(text, str):
            text = str(text)
        
        # Обрезаем текст до 512 символов
        truncated_text = text[:512]
        
        # Применяем классификатор с return_all_scores=True
        results = classifier(truncated_text, truncation=True)
        
        # Преобразуем список результатов в словарь
        scores = {result['label']: result['score'] for result in results[0]}
        return scores
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return {"positive": 0, "neutral": 0, "negative": 0}

In [ ]:
emotional_gdf = gpd.read_file(gpd_path)

In [ ]:
# Применяем функцию ко всем строкам GeoDataFrame
emotional_gdf['emotion_scores'] = emotional_gdf['text'].apply(classify_text)

# Преобразуем словари с вероятностями в отдельные столбцы
emotion_df = pd.json_normalize(emotional_gdf['emotion_scores'])

# Объединяем вероятности с исходным GeoDataFrame
emotional_gdf = emotional_gdf.join(emotion_df)

In [ ]:
# Сгруппируем данные по паркам
grouped = emotional_gdf.groupby('name').agg({
    'positive': 'sum',
    'neutral': 'sum',
    'negative': 'sum',
    'geometry': 'first'  # Используем первую геометрию (или объединение, если нужно)
})

# Нормализуем вероятности для каждого парка
emotion_columns = ['positive', 'neutral', 'negative']
grouped[emotion_columns] = grouped[emotion_columns].div(grouped[emotion_columns].sum(axis=1), axis=0)

# Преобразуем результат обратно в GeoDataFrame
grouped_gdf = gpd.GeoDataFrame(grouped, geometry='geometry')

# Сохраняем результат в файл GeoJSON
grouped_gdf.to_file("output_with_emotional_scores.geojson", driver="GeoJSON")

print("Анализ завершён. Итоговые данные сохранены в 'output_with_emotional_scores.geojson'.")